In [1]:
#coding:utf-8

In [2]:
import sys
import pandas as pd
import numpy as np
all_datasets_path = '../../Desktop/datasets/'
first_train_data_path = '../../Desktop/datasets/train/first_train_data_'
first_test_data_path = '../../Desktop/datasets/test/first_test_data_'
second_train_data_path = '../../Desktop/datasets/train/second_train_data_'
second_test_data_path = '../../Desktop/datasets/test/second_test_data_'
third_train_data_path = '../../Desktop/datasets/train/third_train_data_'
third_test_data_path = '../../Desktop/datasets/test/third_test_data_'

In [3]:
register = pd.read_csv('../../Desktop/datasets/user_register_log.txt',
                       sep='\t',names =['user_id','re_day','re_type','de_type'])

In [4]:
register.head()

,user_id,re_day,re_type,de_type
0,744025,1,1,283
1,1270299,1,1,259
2,571220,1,1,2
3,1308501,1,0,23
4,745554,1,2,0


In [5]:
activity = pd.read_csv('../../Desktop/datasets/user_activity_log.txt',
                       sep='\t',names =['user_id','day','page','video_id','author_id', 'action_type'])
activity.head()

,user_id,day,page,video_id,author_id,action_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


In [6]:
video= pd.read_csv('../../Desktop/datasets/video_create_log.txt',sep='\t',names=['user_id','day'])

In [7]:
video.head()

,user_id,day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


In [8]:
launch = pd.read_csv('../../Desktop/datasets/app_launch_log.txt',sep='\t',names = ['user_id','day'])
launch.head()

,user_id,day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [13]:
def cut_as_time(new_datasets_path,start_day,end_day):
    
    temp_launch = launch[(launch['day']>=start_day)&(launch['day']<=end_day)]
    temp_video = video[(video['day']>=start_day)&(video['day']<=end_day)]
    temp_register =register[(register['re_day']>=start_day)&(register['re_day']<=end_day)]
    temp_activity = activity[(activity['day']>=start_day)&(activity['day']<=end_day)]
    
    temp_launch.to_csv(new_datasets_path+'launch.csv',index= False)
    temp_activity.to_csv(new_datasets_path+'activity.csv',index= False)
    temp_register.to_csv(new_datasets_path+'register.csv',index= False)
    temp_video.to_csv(new_datasets_path+'video.csv',index= False)
    print('根据起始时间结束时间将数据集取出，存入——————',new_datasets_path)
    

In [15]:
def generate_datasets():
    print('开始划分数据集-----')
    #1-16天预测17-23天的活跃用户
 
    cut_as_time(first_train_data_path,1,16)
    cut_as_time(first_test_data_path,17,23)
    print('第一个数据集(16-20天预测17-23天）（训练、测试）划分完成')
     #17-23天预测24-30天的活跃用户
    cut_as_time(second_train_data_path,17,23)
    cut_as_time(second_test_data_path,24,30)
    print('第二个数据集（17-23天预测24-30天）（训练、测试）划分完成')
     #15-30天预测31-37天的活跃用户
    
    cut_as_time(third_train_data_path,15,30)
    print('第三个数据集（15-30预测31-37天）（训练）划分完成')
    print('所有数据集划分完成///////////')
    

In [18]:
generate_datasets()

开始划分数据集-----
根据起始时间结束时间将数据集取出，存入—————— ../../Desktop/datasets/train/first_train_data_
根据起始时间结束时间将数据集取出，存入—————— ../../Desktop/datasets/test/first_test_data_
第一个数据集(16-20天预测17-23天）（训练、测试）划分完成
根据起始时间结束时间将数据集取出，存入—————— ../../Desktop/datasets/train/second_train_data_
根据起始时间结束时间将数据集取出，存入—————— ../../Desktop/datasets/test/second_test_data_
第二个数据集（17-23天预测24-30天）（训练、测试）划分完成
根据起始时间结束时间将数据集取出，存入—————— ../../Desktop/datasets/train/third_train_data_
第三个数据集（15-30预测31-37天）（训练）划分完成
所有数据集划分完成///////////


In [9]:
#生成特征部分


In [10]:
import sys
from collections import Counter
from scipy.stats import stats
import numpy as np


In [11]:
train_path = 'trainNtest/train.csv'
test_path = 'trainNtest/test.csv'

In [12]:
#构建训练集与测试集和特征
#获取所有ID 查看对应ID是否在测试集中出现过
def get_train_label(train_path,test_path):
    
    #合并
    train_reg = pd.read_csv(train_path+'register.csv',usecols=['user_id'])
    train_video = pd.read_csv(train_path+'video.csv',usecols=['user_id'])
    train_lau = pd.read_csv(train_path+'launch.csv',usecols['user_id'])
    train_act = pd.read_csv(train_path+'activity.csv',usecols['user_id'])
    train_data_id = np.unique(pd.concat([train_reg,train_video,train_lau,train_act]))
    
    test_reg = pd.read_csv(test_path+'register.csv',usecols=['user_id'])
    test_video = pd.read_csv(test_path+'video.csv',usecols=['user_id'])
    test_lau = pd.read_csv(test_path+'launch.csv',usecols['user_id'])
    test_act = pd.read_csv(test_path+'activity.csv',usecols['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_video,test_lau,test_act]))
    
    train_label = []#构建一个空的列表
    for i in train_data_id:
        if i in test_data_id:
            train_label.append(1)
        else:
            train_label.append(0)
            
            
    train_data = pd.DataFrame()
    train_data['user_id'] = train_data_id
    train_data['label'] = train_label
    return train_data

#获取测试集用户

def get_test(test_path):
    test_reg = pd.read_csv(test_path+'register.csv',usecols=['user_id'])
    test_video = pd.read_csv(test_path+'video.csv',usecols=['user_id'])
    test_lau = pd.read_csv(test_path+'launch.csv',usecols['user_id'])
    test_act = pd.read_csv(test_path+'activity.csv',usecols['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_video,test_lau,test_act]))
    
    test_data = pd.DataFrame()
    test_data['user_id'] = test_data_id
    return test_data


#获取特征值    


#1.注册表用户特征
def get_reg_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature


#2.拍摄表用户特征
def get_video_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

#3.行为表用户特征
def get_act_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature


#4.启动表用户特征
def get_lau_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature
        
    
#处理特征
def deal_feature(path,user_id):
    reg = pd.read_csv(path+register)
    lau = pd.read_csv(path+launch)
    act = pd.read_csv(path+activity)
    vid = pd.read_csv(path+video)
    feature = pd.DataFrame()
    feature['user_id'] = user_id

    video['max_day'] = np.max(reg['register_day'])
    video_feature = vid.groupby('user_id',sort = True).apply(get_video_feature)
    feature = pd.merge(feature,pd.DataFrame(video_feature),on='user_id',how= 'left')
    print('拍摄日志表提取特征完毕')
    
    
    #注册表
    reg['max_day'] = np.max(reg['register_day'])
    reg_feature = reg.groupby('user_id',sort = True).apply(get_reg_feature)
    feature = pd.merge(feature,pd.DataFrame(reg_feature),on='user_id',how= 'left')
    print('注册日志表提取特征完毕')
    
    
    #行为表
    act['max_day'] = np.max(reg['register_day'])
    act_feature = act.groupby('user_id',sort = True).apply(get_act_feature)
    feature = pd.merge(feature,pd.DataFrame(act_feature),on='user_id',how= 'left')
    print('行为日志表提取特征完毕')
    
    
    #启动日志
    lau['max_day'] = np.max(reg['register_day'])
    lau_feature = lau.groupby('user_id',sort = True).apply(get_lau_feature)
    feature = pd.merge(feature,pd.DataFrame(lau_feature),on='user_id',how= 'left')
    print('启动日志表提取特征完毕')
    

In [ ]:
##one_dataSet_train_path = 'one_dataSet_train_'
#one_dataSet_test_path = 'one_dataSet_test_'
#two_dataSet_train_path = 'two_dataSet_train_'
#two_dataSet_test_path = 'two_dataSet_test_'
#three_dataSet_train_path = 'three_dataSet_train_'

train_path = 'train.csv'
test_path = 'test.csv'

register = 'register.csv'
video = 'video.csv'
launch = 'launch.csv'
activity = 'activity.csv'

# 构建训练集与测试集与特征
# 获取所有id 查看对应id是否在测试集里面出现过
def get_train_label(train_path,test_path):
    train_reg = pd.read_csv(train_path + register,usecols = ['user_id'])
    train_vid = pd.read_csv(train_path + video,usecols = ['user_id'])
    train_lau = pd.read_csv(train_path + launch,usecols = ['user_id'])
    train_act = pd.read_csv(train_path + activity,usecols = ['user_id'])
    train_data_id = np.unique(pd.concat([train_reg,train_vid,train_lau,train_act]))
    
    test_reg = pd.read_csv(test_path + register,usecols = ['user_id'])
    test_vid = pd.read_csv(test_path + video,usecols = ['user_id'])
    test_lau = pd.read_csv(test_path + launch,usecols = ['user_id'])
    test_act = pd.read_csv(test_path + activity,usecols = ['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_vid,test_lau,test_act]))
    
    train_label = []
    for i in train_data_id:
        for i in test_data_id:
            train_label.append(1)
        else:
            train_label.append(0)
    train_data = pd.DataFrame()
    train_data['user_id'] = train_data_id
    train_data['label'] = train_label
    return train_data

def get_test(test_path):
    test_reg = pd.read_csv(test_path + register,usecols = ['user_id'])
    test_vid = pd.read_csv(test_path + video,usecols = ['user_id'])
    test_lau = pd.read_csv(test_path + launch,usecols = ['user_id'])
    test_act = pd.read_csv(test_path + activity,usecols = ['user_id'])
    test_data_id = np.unique(pd.concat([test_reg,test_vid,test_lau,test_act]))
    test_data = pd.DataFrame()
    test_data['user_id'] = test_data_id
    return test_data

def get_video_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_register_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_launch_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def get_activity_feature(row):
    feature = pd.Series()
    feature['user_id'] = list(row['user_id'])[0]
    return feature

def deal_feature(path,user_id):
    reg = pd.read_csv(path + register)
    vid = pd.read_csv(path + video)
    lau = pd.read_csv(path + launch)
    act = pd.read_csv(path + activity)
    feature = pd.DataFrame()
    feature['user_id'] = user_id
    
    # w...
    vid['max_day'] = np.max(reg['register_day'])
    vid_feature = vid.groupby('user_id',sort=Ture).apply(get_video_feature)
    feature = pd.merge(feature,pd.DataFrame(video_feature),on='user_id',how='left')
    print("video表特征提取完毕")
    
    # w...
    reg['max_day'] = np.max(reg['register_day'])
    reg_feature = reg.groupby('user_id',sort=Ture).apply(get_register_feature)
    feature = pd.merge(feature,pd.DataFrame(reg_feature),on='user_id',how='left')
    print("register表特征提取完毕")
    
    # w...
    lau['max_day'] = np.max(reg['register_day'])
    lau_feature = lau.groupby('user_id',sort=Ture).apply(get_launch_feature)
    feature = pd.merge(feature,pd.DataFrame(lau_feature),on='user_id',how='left')
    print("launch表特征提取完毕")
    
    # w......
    act['max_day'] = np.max(reg['register_day'])
    act_feature = act.groupby('user_id',sort=Ture).apply(get_activity_feature)
    feature = pd.merge(feature,pd.DataFrame(act_feature),on='user_id',how='left')
    print("activity表特征提取完毕")
    return feature

# 构建特征
def get_data_feature():
    first_train_data = get_train_label(first_train_data_path,first_test_data_path)
    first_feature = deal_feature(first_train_data_path,first_train_data['user_id'])
    first_feature['label'] = first_train_data['label']
    print("第一组训练数据特征提取完毕")
    
    second_train_data = get_train_label(second_train_data_path,second_test_data_path)
    second_feature = deal_feature(second_train_data_path,second_test_data_path['user_id'])
    second_feature['label'] = second_train_data['label']
    print("第二组训练数据特征提取完毕")
    
    train_feature = pd.concat([first_feature,second_feature])
    train_feature.to_csv(train_path,index=False)
    print("训练数据存储完毕")
    
    test_data = get_test(third_data_train_path)
    test_feature = deal_feature(third_train_data_path,test_data['user_id'])
    test_feature.to_csv(test_path,index=False)
    print("测试数据存储完毕")

get_data_feature()